Importing Libraries

In [ ]:
!pip install spacy
import spacy
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 20000000

Importing and Splitting Large Text Corpus into Sentences

In [ ]:
with open('/content/Scraped_Articles_Large_Text_Corpus', 'r', encoding="utf-8", errors="ignore") as f:
    sentences_articles = f.read()
about_doc = nlp(sentences_articles)
unclean_sentences = list(about_doc.sents) 

In [ ]:
sentences = [x for x in unclean_sentences if len (x) > 2]
sentences[:10]
sorted(sentences, key=len)
len(sentences)

Transform Sentences into Vectors

In [ ]:
embedder = SentenceTransformer('all-mpnet-base-v2')
corpus = sentences
corpus_embeddings = embedder.encode(corpus, show_progress_bar=True, batch_size = 64)
print(corpus_embeddings)

Insert Anchor Sentences

In [ ]:
queries = ["This is a technocratic anchor sentence", \
           "This is another technocratic anchor sentence",\
           "This is a third technocratic anchor sentence",\
           ]

In [ ]:
top_k =min(10, len(corpus))

Finding 10 Most Semantically Similar Sentences in Large Text Corpus for Each Anchor Sentence

In [ ]:
for query in queries:
  query_embedding = embedder.encode(query, convert_to_tensor = True)
  cos_scores = util.cos_sim(query_embedding, corpus_embeddings) [0]
  top_results = torch.topk(cos_scores, k=top_k)
  print("\n\nQuery ==> ", query)
  print('-----------------------------------------------')

  for score, idx in zip(top_results[0], top_results[1]) :
    print(corpus[idx], "(Score: (:.4f))".format(score))